In [193]:
# dependencies
import requests
import json
import pandas as pd
import time
import os

# import spotipy
# !pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [194]:
from config import spotify_client_ID as sp_client, spotify_client_secret as sp_secret

In [195]:
# fill-in mini dataset: current top 10 charting songs
# https://www.americantop40.com/charts/top-40-238/october-8-2022/

top10 = pd.read_csv("../00_data/top10.csv")
top10

,week,position,song,artist,features_artist,URI
0,8-Oct-22,1,I Like You (A Happier Song),Post Malone,Doja Cat,spotify:track:7JyFwJC3uRpiNfq7oKafgn
1,8-Oct-22,2,Sunroof,Nicky Youre,Dazy,spotify:track:4h4QlmocP3IuwYEj2j14p8
2,8-Oct-22,3,Late Night Talking,Harry Styles,NaN,spotify:track:1qEmFfgcLObUfQm0j1W2CK
3,8-Oct-22,4,About Damn Time,Lizzo,NaN,spotify:track:1PckUlxKqWQs3RlWXVBLw3
4,8-Oct-22,5,As It Was,Harry Styles,NaN,spotify:track:4Dvkj6JhhA12EX05fT7y2e
5,8-Oct-22,6,Vegas,Doja Cat,NaN,spotify:track:0hquQWY3xvYqN4qtiquniF
6,8-Oct-22,7,I Ain't Worried,OneRepublic,NaN,spotify:track:4h9wh7iOZ0GGn8QVp4RAOB
7,8-Oct-22,8,Bad Habit,Steve Lacy,NaN,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK
8,8-Oct-22,9,Glimpse of Us,Joji,NaN,spotify:track:6xGruZOHLs39ZbVccQTuPZ
9,8-Oct-22,10,So Good,Halsey,NaN,spotify:track:6kxaaIeowajN7w21PfMLbu


### Get track audio features from Spotify

In [196]:
# set up client credentials 
# https://spotipy.readthedocs.io/en/master/?highlight=spotifyclientcredentials#client-credentials-flow
# API info: https://developer.spotify.com/documentation/web-api/reference/#/

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(sp_client, sp_secret))

In [5]:
# API info: https://developer.spotify.com/documentation/web-api/
## https://developer.spotify.com/documentation/web-api/reference/#/

sp_root_url = 'https://api.spotify.com/v1/'

In [197]:
# function to get audio features
# input a dataframe of top 40 data

def get_spotify_audio_features(top40_df):
        
    # get list of all track URIs
    URIs = top40_df['URI']
    
    # get audio features
    audio_features = sp.audio_features(URIs)
    
    # empty list of genres
    top40_genres = []
    
    for i in range(len(URIs)):
        
        # get the artist's URI
        artist = sp.track(URIs[i])['album']['artists'][0]['uri']
        
        # get the artist's genre(s)
        artist_genres = sp.artist(artist)['genres']
        
        for genre in artist_genres:
            if genre not in top40_genres:
                top40_genres.append(genre)
        
        # get the audio features for each track
        audio_features = sp.audio_features(URIs[i])
        
        # save output to the input dataframe
        #top40_df['danceability'][i] = audio_features[i].get('danceability')
        
    return audio_features

In [207]:
top40_df = pd.read_csv("../00_data/top10.csv")

# list of all track URIs
URIs = top40_df['URI']

# empty list to hold all artist URIs
artist_URIs = []

for i in range(len(URIs)):
    
    # get each artist's URI
    artist_URIs.append(sp.track(URIs[i])['album']['artists'][0]['uri'])

# save artist URIs to the dataframe
top40_df['artist_URI'] = artist_URIs


# empty list and dataframe of genres
top40_genres = []

# empty dictionary to hold all genres associated with each artist
artist_genre_dict = {}

# get genres associated with all artists    
for artist in artist_URIs:
    
    # get the artist's genre(s)
    artist_genres = sp.artist(artist)['genres']
    
    # format genre strings
    artist_genres = [x.replace(' ', '_').lower() for x in artist_genres]
    
    artist_genre_dict[artist] = artist_genres
    
    # save new genres to the list of all top40 genres
    for genre in artist_genres:
        if genre not in top40_genres:
            top40_genres.append(genre)

            
# fill the new genre dataframe
top40_genres_dict = {}
for genre in top40_genres:
    top40_genres_dict[genre] = [0] * len(top40_df)

# create dataframe with genre data, sort df columns
genre_df = pd.DataFrame(top40_genres_dict)
genre_df.columns = sorted(genre_df.columns.tolist())
genre_df['artist'] = artist_URIs
genre_df.index = artist_URIs

# complete genre data
#for artist in artist_URIs:
#    
#    # get the artist's genre(s)
#    artist_genres = sp.artist(artist)['genres']
#    
#    # save new genres to the list of all top40 genres
#    for genre in artist_genres:
#        
#        for i in len(range(genre_df.columns)):
#            if genre in genre_df.columns[i]:
#                top40_genres.append(genre)

#                 
for column in genre_df:
    
    for artist in genre_df.index:
        
        for genre in artist_genre_dict[artist]:
            
            if genre == column:
            
                genre_df.loc[artist, column] = 1

# concatenate all data into one dataframe
#full_df = pd.concat([top40_df,genre_df], axis=1, ignore_index = True)
#full_df
genre_df

,afrofuturism,dance_pop,dfw_rap,electropop,escape_room,etherpop,indie_poptimism,melodic_rap,minnesota_hip_hop,piano_rock,pop,pop_rock,rap,trap_queen,viral_pop,artist
spotify:artist:246dkjvS1zLTtiykXe5h60,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,spotify:artist:246dkjvS1zLTtiykXe5h60
spotify:artist:7qmpXeNz2ojlMl2EEfkeLs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,spotify:artist:7qmpXeNz2ojlMl2EEfkeLs
spotify:artist:6KImCVD70vtIoJWnq6nGn3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,spotify:artist:6KImCVD70vtIoJWnq6nGn3
spotify:artist:56oDRnqbIiwx4mymNEv7dS,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,spotify:artist:56oDRnqbIiwx4mymNEv7dS
spotify:artist:6KImCVD70vtIoJWnq6nGn3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,spotify:artist:6KImCVD70vtIoJWnq6nGn3
spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5
spotify:artist:5Pwc4xIPtQLFEnJriah9YJ,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,spotify:artist:5Pwc4xIPtQLFEnJriah9YJ
spotify:artist:57vWImR43h4CaDao012Ofp,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,spotify:artist:57vWImR43h4CaDao012Ofp
spotify:artist:3MZsBdqDrRTJihTHQrO6Dq,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,spotify:artist:3MZsBdqDrRTJihTHQrO6Dq
spotify:artist:26VFTg2z8YR0cCuwLzESi2,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,spotify:artist:26VFTg2z8YR0cCuwLzESi2


In [ ]:
# complete genre data
for artist in artist_URIs:
    # get the artist's genre(s)
    artist_genres = sp.artist(artist)['genres']
    
    # save new genres to the list of all top40 genres
    for genre in artist_genres:
        
        for i in len(range(genre_df.columns)):
            if genre in genre_df.columns[i]:
                top40_genres.append(genre)

for column in genre_df:
    
    for artist in genre_df.index:
        
        for genre in artist_genre_dict[artist]:
            print(genre)
            print(artist)
            if genre == column:
            
                genre_df.loc[artist, column] = 1
# concatenate all data into one dataframe
full_df = pd.concat([top40_df,genre_df],axis=1)


In [208]:
genre_df['artist']

spotify:artist:246dkjvS1zLTtiykXe5h60    spotify:artist:246dkjvS1zLTtiykXe5h60
spotify:artist:7qmpXeNz2ojlMl2EEfkeLs    spotify:artist:7qmpXeNz2ojlMl2EEfkeLs
spotify:artist:6KImCVD70vtIoJWnq6nGn3    spotify:artist:6KImCVD70vtIoJWnq6nGn3
spotify:artist:56oDRnqbIiwx4mymNEv7dS    spotify:artist:56oDRnqbIiwx4mymNEv7dS
spotify:artist:6KImCVD70vtIoJWnq6nGn3    spotify:artist:6KImCVD70vtIoJWnq6nGn3
spotify:artist:5cj0lLjcoR7YOSnhnX0Po5    spotify:artist:5cj0lLjcoR7YOSnhnX0Po5
spotify:artist:5Pwc4xIPtQLFEnJriah9YJ    spotify:artist:5Pwc4xIPtQLFEnJriah9YJ
spotify:artist:57vWImR43h4CaDao012Ofp    spotify:artist:57vWImR43h4CaDao012Ofp
spotify:artist:3MZsBdqDrRTJihTHQrO6Dq    spotify:artist:3MZsBdqDrRTJihTHQrO6Dq
spotify:artist:26VFTg2z8YR0cCuwLzESi2    spotify:artist:26VFTg2z8YR0cCuwLzESi2
Name: artist, dtype: object

In [201]:
genre_df.drop(axis=0)

ValueError: Need to specify at least one of 'labels', 'index' or 'columns'

In [187]:
artist_genre_dict

{'spotify:artist:246dkjvS1zLTtiykXe5h60': ['dfw_rap', 'melodic_rap', 'rap'],
 'spotify:artist:7qmpXeNz2ojlMl2EEfkeLs': [],
 'spotify:artist:6KImCVD70vtIoJWnq6nGn3': ['pop'],
 'spotify:artist:56oDRnqbIiwx4mymNEv7dS': ['escape_room',
  'minnesota_hip_hop',
  'pop',
  'trap_queen'],
 'spotify:artist:5cj0lLjcoR7YOSnhnX0Po5': ['dance_pop', 'pop'],
 'spotify:artist:5Pwc4xIPtQLFEnJriah9YJ': ['piano_rock', 'pop', 'pop_rock'],
 'spotify:artist:57vWImR43h4CaDao012Ofp': ['afrofuturism', 'pop'],
 'spotify:artist:3MZsBdqDrRTJihTHQrO6Dq': ['viral_pop'],
 'spotify:artist:26VFTg2z8YR0cCuwLzESi2': ['dance_pop',
  'electropop',
  'etherpop',
  'indie_poptimism',
  'pop']}

In [174]:
'dfw_rap' == genre_df.columns[2]

True

In [181]:
for column in genre_df.iteritems():
    print(column[1])

spotify:artist:246dkjvS1zLTtiykXe5h60    0
spotify:artist:7qmpXeNz2ojlMl2EEfkeLs    0
spotify:artist:6KImCVD70vtIoJWnq6nGn3    0
spotify:artist:56oDRnqbIiwx4mymNEv7dS    0
spotify:artist:6KImCVD70vtIoJWnq6nGn3    0
spotify:artist:5cj0lLjcoR7YOSnhnX0Po5    0
spotify:artist:5Pwc4xIPtQLFEnJriah9YJ    0
spotify:artist:57vWImR43h4CaDao012Ofp    0
spotify:artist:3MZsBdqDrRTJihTHQrO6Dq    0
spotify:artist:26VFTg2z8YR0cCuwLzESi2    0
Name: afrofuturism, dtype: int64
spotify:artist:246dkjvS1zLTtiykXe5h60    0
spotify:artist:7qmpXeNz2ojlMl2EEfkeLs    0
spotify:artist:6KImCVD70vtIoJWnq6nGn3    0
spotify:artist:56oDRnqbIiwx4mymNEv7dS    0
spotify:artist:6KImCVD70vtIoJWnq6nGn3    0
spotify:artist:5cj0lLjcoR7YOSnhnX0Po5    0
spotify:artist:5Pwc4xIPtQLFEnJriah9YJ    0
spotify:artist:57vWImR43h4CaDao012Ofp    0
spotify:artist:3MZsBdqDrRTJihTHQrO6Dq    0
spotify:artist:26VFTg2z8YR0cCuwLzESi2    0
Name: dance_pop, dtype: int64
spotify:artist:246dkjvS1zLTtiykXe5h60    0
spotify:artist:7qmpXeNz2ojlMl2EEfk

In [49]:
# get audio features of the current top 10 charting songs
top10_features = get_spotify_audio_features(top10)

In [41]:
top10_features

[{'danceability': 0.301,
  'energy': 0.303,
  'key': 4,
  'loudness': -12.39,
  'mode': 1,
  'speechiness': 0.0458,
  'acousticness': 0.177,
  'instrumentalness': 0.031,
  'liveness': 0.0979,
  'valence': 0.0384,
  'tempo': 141.997,
  'type': 'audio_features',
  'id': '7JyFwJC3uRpiNfq7oKafgn',
  'uri': 'spotify:track:7JyFwJC3uRpiNfq7oKafgn',
  'track_href': 'https://api.spotify.com/v1/tracks/7JyFwJC3uRpiNfq7oKafgn',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7JyFwJC3uRpiNfq7oKafgn',
  'duration_ms': 145796,
  'time_signature': 4},
 {'danceability': 0.768,
  'energy': 0.714,
  'key': 10,
  'loudness': -5.11,
  'mode': 1,
  'speechiness': 0.0401,
  'acousticness': 0.352,
  'instrumentalness': 0,
  'liveness': 0.15,
  'valence': 0.842,
  'tempo': 131.443,
  'type': 'audio_features',
  'id': '4h4QlmocP3IuwYEj2j14p8',
  'uri': 'spotify:track:4h4QlmocP3IuwYEj2j14p8',
  'track_href': 'https://api.spotify.com/v1/tracks/4h4QlmocP3IuwYEj2j14p8',
  'analysis_url': 'https://api.s

In [52]:
f1 = sp.audio_features(top10.URI[0])
f1

[{'danceability': 0.301,
  'energy': 0.303,
  'key': 4,
  'loudness': -12.39,
  'mode': 1,
  'speechiness': 0.0458,
  'acousticness': 0.177,
  'instrumentalness': 0.031,
  'liveness': 0.0979,
  'valence': 0.0384,
  'tempo': 141.997,
  'type': 'audio_features',
  'id': '7JyFwJC3uRpiNfq7oKafgn',
  'uri': 'spotify:track:7JyFwJC3uRpiNfq7oKafgn',
  'track_href': 'https://api.spotify.com/v1/tracks/7JyFwJC3uRpiNfq7oKafgn',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7JyFwJC3uRpiNfq7oKafgn',
  'duration_ms': 145796,
  'time_signature': 4}]

In [62]:
f2 = sp.track(top10.URI[0])
f2['album']['artists'][0]['uri']

'spotify:artist:246dkjvS1zLTtiykXe5h60'

In [13]:
requests.get('https://api.spotify.com/v1/audio-features/7JyFwJC3uRpiNfq7oKafgn')

<Response [401]>

In [55]:
sp.artist('spotify:artist:1it6OQb9fceS1M3gfd6erX')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1it6OQb9fceS1M3gfd6erX'},
 'followers': {'href': None, 'total': 51273},
 'genres': ['dark jazz'],
 'href': 'https://api.spotify.com/v1/artists/1it6OQb9fceS1M3gfd6erX',
 'id': '1it6OQb9fceS1M3gfd6erX',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273471b29b7bb824ed3b930b538',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02471b29b7bb824ed3b930b538',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d00004851471b29b7bb824ed3b930b538',
   'width': 64}],
 'name': 'The Kilimanjaro Darkjazz Ensemble',
 'popularity': 30,
 'type': 'artist',
 'uri': 'spotify:artist:1it6OQb9fceS1M3gfd6erX'}

In [12]:
requests.get('https://accounts.spotify.com/api/token')

<Response [405]>

In [18]:
sp